In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import *
from scipy.stats import norm,kstest,shapiro,t
import warnings
import seaborn as sns
import statsmodels.api as sm
import statistics
warnings.filterwarnings('ignore')
import scipy as sp
import os
lib_path = os.path.join(os.path.abspath('.'), '..')
sys.path.append(lib_path)
from risk_management import *
from tabulate import tabulate
# Import plotly express for EF plot
import plotly.express as px
px.defaults.template = "plotly_dark"
px.defaults.width = 1000
px.defaults.height = 600
import scipy.optimize as sco

# Problem 1

Part 1 – Calculate the maximum SR portfolio using the method from last week’s homework for the following stocks: AAPL, MSFT, BRK-B, CSCO, and JNJ.

Use the returns from the end of the history (1-14) until the end of February.

Calculate the Ex-Post Return Attribution for each Stock.

In [3]:
ff_3 = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
ff_4th = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date')

ff_4 = ff_3.join(ff_4th, how='right') / 100
ff_4.rename(columns = {'Mom   ':"Mom"},inplace=True)
all_rets = pd.read_csv('DailyReturn.csv', parse_dates=['Date']).set_index('Date')
stocks = ['AAPL','MSFT','BRK-B','CSCO','JNJ']
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']

reg_dataset = all_rets[stocks].join(ff_4)
reg_dataset.head()

,AAPL,MSFT,BRK-B,CSCO,JNJ,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,,,,,,
2021-10-21,0.001474,0.010897,-0.002330,-0.009075,-0.002320,0.0037,0.0020,-0.0098,0.0,0.0003
2021-10-22,-0.005285,-0.005149,0.008437,-0.010415,0.001958,-0.0025,-0.0023,0.0102,0.0,0.0031
2021-10-25,-0.000336,-0.003332,0.003527,0.000544,0.002199,0.0058,0.0049,-0.0016,0.0,0.0124
2021-10-26,0.004575,0.006426,0.002033,0.012151,0.010178,0.0004,-0.0071,-0.0032,0.0,-0.0022
2021-10-27,-0.003148,0.042114,-0.010555,0.006630,-0.012127,-0.0076,-0.0074,-0.0119,0.0,-0.0008


In [4]:
param_dict = {}
avg_daily_rets = pd.Series(dtype='float64')
avg_factor_rets = ff_4.loc['2012-1-14':'2022-1-14'].mean(axis=0)

x1 = reg_dataset['Mkt-RF']
x2 = reg_dataset['SMB']
x3 = reg_dataset['HML']
x4 = reg_dataset['Mom']

X = pd.DataFrame({"Mkt-RF":x1, "SMB":x2,"HML":x3,'Mom':x4})
X_ = sm.add_constant(X)
for st in stocks:
    Y = reg_dataset[st] - reg_dataset['RF']
    result = sm.OLS(Y, X_).fit()
    param_dict[st] = (result.params)
    
    avg_daily_rets[st] = (result.params[factors] * avg_factor_rets[factors]).sum() \
                            + avg_factor_rets['RF'] 
    
    
# 2. geometric annual returns: mean and covariance
geo_means = np.log(1 + avg_daily_rets)*252
geo_covariance = np.log(1 + all_rets[stocks]).cov()*252
weights=effFrontier.tangency_weights(geo_means,geo_covariance,0.0075)
weights

,Weight
AAPL,0.100775
MSFT,0.209163
BRK-B,0.438186
CSCO,0.081112
JNJ,0.170764


In [5]:
updated_prices = pd.read_csv("updated_prices.csv", parse_dates=['Date']).set_index('Date').iloc[:,1:]
updated_rets = updated_prices.pct_change().dropna()
total_returns = updated_prices.iloc[-1]/updated_prices.iloc[0]-1
total_returns

AAPL    -0.044720
MSFT    -0.034791
BRK-B   -0.008268
CSCO    -0.091102
JNJ     -0.013189
dtype: float64

In [6]:
u_weights = attribution.update_weights(weights.T, updated_rets)
weighted_rets = u_weights * updated_rets.values


ReturnAttribution = attribution.return_attribution(weighted_rets)
VolAttribution = attribution.risk_attribution(weighted_rets)


AttributionTable = pd.DataFrame([total_returns.values,ReturnAttribution,VolAttribution],columns = stocks,\
                               index = ["Total Return","Return Attribution","Vol Attribution"])
AttributionTable["Portfolio"] = np.array([(weights.T*total_returns).sum(axis=1)[0],
                                        AttributionTable.iloc[1].sum(),
                                        AttributionTable.iloc[2].sum()])

round(AttributionTable,4)

,AAPL,MSFT,BRK-B,CSCO,JNJ,Portfolio
Total Return,-0.0447,-0.0348,-0.0083,-0.0911,-0.0132,-0.0250
Return Attribution,-0.0046,-0.0071,-0.0037,-0.0075,-0.0022,-0.0250
Vol Attribution,0.0016,0.0032,0.0048,0.0009,0.0014,0.0119


# Problem 2
Using the same data as Problem 1, attribute realized risk and return to the Fama French 3+Momentum model. Report the residual total as Portfolio Alpha.

In [7]:
updated_ff_3 = pd.read_csv('updated_F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
updated_ff_4th = pd.read_csv('updated_F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date')
updated_ff_4 = updated_ff_3.join(updated_ff_4th, how='right') / 100
updated_ff_4.rename(columns = {'Mom   ':"Mom"},inplace=True)
updated_factor_rets = pd.concat([ff_4, updated_ff_4]).loc[updated_rets.index, factors]
updated_factor_rets.head()

,Mkt-RF,SMB,HML,Mom
Date,,,,
2022-01-18,-0.0208,-0.0102,0.0139,0.0014
2022-01-19,-0.0101,-0.0032,-0.0066,-0.0043
2022-01-20,-0.0115,-0.0086,-0.0022,-0.0044
2022-01-21,-0.0209,0.0029,0.0048,0.0042
2022-01-24,0.0053,0.0181,-0.0028,0.0004


In [8]:
param_dict.keys()
betas = np.zeros((len(param_dict.keys()),4))
for i, st in enumerate(param_dict):
    betas[i] =  param_dict[st][factors]
   
    

u_factor_weight = u_weights@betas
u_factor_weight.columns = factors
weighted_factor_rets = u_factor_weight* updated_factor_rets.values
alpha_rets = weighted_rets.sum(axis=1) - weighted_factor_rets.sum(axis=1)
weighted_factor_rets["Alpha"] = alpha_rets
weighted_factor_rets

,Mkt-RF,SMB,HML,Mom,Alpha
Date,,,,,
2022-01-18,-0.016009,0.004353,0.001683,-0.000109,-0.005012
2022-01-19,-0.007763,0.001366,-0.000815,0.000342,-0.004030
2022-01-20,-0.008841,0.003682,-0.000265,0.000346,-0.004840
2022-01-21,-0.016068,-0.001243,0.000574,-0.000329,0.001468
2022-01-24,0.004070,-0.007764,-0.000334,-0.000032,-0.000035
2022-01-25,-0.010991,0.002574,0.003303,-0.000203,0.007422
2022-01-26,-0.002298,0.004628,0.000113,-0.001164,0.007185
2022-01-27,-0.005980,0.007003,0.000944,-0.000245,-0.000680
2022-01-28,0.018775,0.000646,-0.002488,0.000219,0.005581


In [9]:
ReturnAttribution = attribution.return_attribution(weighted_factor_rets)
VolAttribution = attribution.risk_attribution(weighted_factor_rets)

fac_total_returns = (1+updated_factor_rets).prod()-1
AttributionTable = pd.DataFrame([fac_total_returns,ReturnAttribution,VolAttribution],columns = weighted_factor_rets.columns,\
                               index = ["Total Return","Return Attribution","Vol Attribution"])
AttributionTable["Portfolio"] = np.array([(weights.T*total_returns).sum(axis=1)[0],
                                        AttributionTable.iloc[1].sum(),
                                        AttributionTable.iloc[2].sum()])

round(AttributionTable,4)

,Mkt-RF,SMB,HML,Mom,Alpha,Portfolio
Total Return,-0.0580,-0.0011,0.0203,0.0082,NaN,-0.0250
Return Attribution,-0.0445,0.0001,0.0024,-0.0006,0.0176,-0.0250
Vol Attribution,0.0099,-0.0001,-0.0007,0.0001,0.0027,0.0119


# Problem3
Using the same data as Problem 1 and assuming a 0 mean return, fit a t distribution to each stock return series. Simulate the system using a Gaussian Copula. Find the Risk Parity portfolio using ES as the risk measure.

In [10]:
def cal_port_vol(weights, cov):
    w = np.array(weights).flatten()
    port_std = np.sqrt(w@cov@w)
    return port_std

def cal_component_std(weights, cov):
    w = np.array(weights).flatten()
    port_std = cal_port_vol(weights, cov)
    csd = w * (cov@w)/port_std
    return csd

def risk_budget(weights, cov):
    w = np.array(weights).flatten()
    port_std = cal_port_vol(weights, cov)
    csd = cal_component_std(weights, cov)
    return csd/port_std

def cal_csd_sse(weights, cov):
    w = np.array(weights).flatten()
    port_std = cal_port_vol(weights, cov)
    csd = cal_component_std(weights, cov)
    csd_ = csd - np.mean(csd)
    return sum(csd_**2)

def risk_parity_weights(cov):
    numOfAssets = cov.shape[0]
    initial_wts = np.array(numOfAssets * [1. / numOfAssets])
    cons = ({'type': 'eq', 'fun': lambda x: sum(x) - 1})
    cov = np.array(cov)
    bnds = tuple((0, 1) for x in range(numOfAssets))
    opt_wts = sco.minimize(lambda w: 1e5*cal_csd_sse(w, cov), initial_wts,bounds=bnds, constraints=cons)
    return opt_wts.x
    
w2 = np.array([0.138,0.232,0.291,0.147,0.193])
risk_budget(w2,geo_covariance)
risk_parity_weights(geo_covariance)  

array([0.15420279, 0.14256559, 0.26516038, 0.15078617, 0.28728506])

In [24]:
def cal_port_es(weights, rets):
    w = np.array(weights).flatten()
    return ES.ES(rets@w)

def cal_component_es(weights, rets):
    w = np.array(weights).flatten()
    e = 1e-6
    numOfAssets = len(weights)
    es = cal_port_es(w, rets)
    ces = np.zeros(numOfAssets)

    for i in range(numOfAssets):
        weight_i = w[i]
        w[i] += e
        ces[i] = weight_i * (cal_port_es(w, rets) - es) / e
        w[i] -= e
    
    return ces

def cal_ces_sse(weights, rets):
    ces = cal_component_es(weights, rets)
    ces_ = ces - np.mean(ces)
    return ces_@ces_.T

def risk_parity_es_weights(rets):
    numOfAssets = rets.shape[1]
    initial_wts = np.array(numOfAssets * [1. / numOfAssets])
    cons = ({'type': 'eq', 'fun': lambda x: sum(x) - 1})
    rets = np.array(rets)
    bnds = tuple((0, 1) for x in range(numOfAssets))
    opt_wts = sco.minimize(lambda w: 1e5*cal_ces_sse(w, rets), initial_wts,bounds=bnds, constraints=cons)
    return opt_wts.x
    
risk_parity_es_weights(all_rets[stocks])

array([0.18017754, 0.10598453, 0.21127725, 0.27865071, 0.22390998])

In [44]:
def copula_t_simulation(ret, nOfDarws):
    '''
    Params: ret should be the rate of return
    nOfDarws: # simulations
    '''
    n=ret.shape[1]
    stock_cdf = pd.DataFrame()
    t_params = []

    for col in ret.columns:
        ret[col] -= ret[col].mean()
        df, mean, scale = simulation.Return_T_params(ret[col])
        t_params.append([df, mean, scale])
        stock_cdf[col] = t.cdf(ret[col], df=df, loc=mean, scale=scale)

    Corr_spearman = sp.stats.spearmanr(stock_cdf)[0]

    cholesky = psd.chol_psd(Corr_spearman)
    simuNormal = pd.DataFrame(sp.stats.norm.rvs(size=(n,nOfDarws)))
    simulatedT= (cholesky @ simuNormal).T

    Simu_data = pd.DataFrame()
    for i in range(n):
        simu = norm.cdf(simulatedT.iloc[:, i])
        Simu_data[ret.columns[i]] = t.ppf(simu, df=t_params[i][0], loc=t_params[i][1], scale=t_params[i][2])

    return Simu_data


sim_rets = copula_t_simulation(all_rets[stocks],5000)

risk_parity_es_weights(sim_rets)

array([0.15628936, 0.14455667, 0.26134849, 0.14646146, 0.29134403])